# 🏀 NCAA March Madness Feature Engineering: Building Predictive Team Metrics

## 🚀 Objective
This notebook transforms raw NCAA basketball data into **meaningful features** for predicting tournament winners.  
By engineering **team performance metrics**, we prepare a dataset that can be used for **machine learning models**.

## 🔬 Key Steps
1️⃣ **Load & inspect datasets** (season results, tournament games, seeds, teams)  
2️⃣ **Compute team performance metrics** (win percentage, avg points, point differential)  
3️⃣ **Merge team stats into tournament match data**  
4️⃣ **Create feature differences for predictive modeling**  

📌 **Next Step:** Use these engineered features to **train a machine learning model** to predict NCAA tournament outcomes! 🎯


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load datasets
season_results = pd.read_csv("../data/MRegularSeasonCompactResults.csv")  # Regular season games
tourney_results = pd.read_csv("../data/MNCAATourneyCompactResults.csv")  # Tournament games
seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")  # Team seed rankings
teams = pd.read_csv("../data/MTeams.csv")  # Team names

# Display first few rows of each dataset
display(season_results.head(), tourney_results.head(), seeds.head(), teams.head())

# Print column names for reference
print("Season Results Columns:", season_results.columns)
print("Tournament Results Columns:", tourney_results.columns)
print("Seeds Columns:", seeds.columns)
print("Teams Columns:", teams.columns)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025


Season Results Columns: Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT'],
      dtype='object')
Tournament Results Columns: Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT'],
      dtype='object')
Seeds Columns: Index(['Season', 'Seed', 'TeamID'], dtype='object')
Teams Columns: Index(['TeamID', 'TeamName', 'FirstD1Season', 'LastD1Season'], dtype='object')


### 📊 Computing Team Strength
We will calculate:
- **Win Percentage** = Total Wins / Total Games
- **Average Points Scored** (Offensive Strength)
- **Average Points Allowed** (Defensive Strength)
- **Point Differential** = (Avg Points Scored - Avg Points Allowed)


In [5]:
# Calculate total wins per team
win_counts = season_results.groupby("WTeamID").size().reset_index(name="Wins")

# Calculate total games played (wins + losses)
total_games = season_results.groupby("WTeamID").size().add(season_results.groupby("LTeamID").size(), fill_value=0).reset_index(name="TotalGames")

# Merge wins and total games
team_stats = win_counts.merge(total_games, left_on="WTeamID", right_on="WTeamID", how="left")
team_stats["WinPercentage"] = team_stats["Wins"] / team_stats["TotalGames"]

# Compute average points scored (offense) and allowed (defense)
avg_points_scored = season_results.groupby("WTeamID")["WScore"].mean().reset_index(name="AvgPointsScored")
avg_points_allowed = season_results.groupby("LTeamID")["LScore"].mean().reset_index(name="AvgPointsAllowed")

# Merge into team stats
team_stats = team_stats.merge(avg_points_scored, on="WTeamID", how="left")
team_stats = team_stats.merge(avg_points_allowed, left_on="WTeamID", right_on="LTeamID", how="left")

# Compute point differential (offense - defense)
team_stats["PointDifferential"] = team_stats["AvgPointsScored"] - team_stats["AvgPointsAllowed"]

# Drop redundant LTeamID column
team_stats.drop(columns=["LTeamID"], inplace=True, errors="ignore")

# Display updated team stats
display(team_stats.head())


,WTeamID,Wins,TotalGames,WinPercentage,AvgPointsScored,AvgPointsAllowed,PointDifferential
0,1101,151,319,0.473354,76.576159,63.625000,12.951159
1,1102,396,1109,0.357078,70.972222,59.827489,11.144733
2,1103,692,1177,0.587935,76.354046,64.546392,11.807654
3,1104,797,1263,0.631037,77.834379,65.718884,12.115495
4,1105,245,703,0.348506,74.363265,61.403930,12.959335


### 🔢 Adding Seed Information
The **seeding rank** of a team is a strong predictor of tournament performance.  
- Lower **seed values** indicate stronger teams.
- We extract **only the numeric value** of the seed (e.g., `"W01"` → `1`).


In [7]:
# Extract numeric seed value (e.g., "W01" → 1)
seeds["SeedValue"] = seeds["Seed"].apply(lambda x: int(x[1:3]))

# Merge seeds into team stats
team_stats = team_stats.merge(seeds[["TeamID", "SeedValue"]], left_on="WTeamID", right_on="TeamID", how="left")

# Drop redundant TeamID column
team_stats.drop(columns=["TeamID"], inplace=True)

# Display updated team stats
display(team_stats.head())


,WTeamID,Wins,TotalGames,WinPercentage,AvgPointsScored,AvgPointsAllowed,PointDifferential,SeedValue
0,1101,151,319,0.473354,76.576159,63.625000,12.951159,15.0
1,1101,151,319,0.473354,76.576159,63.625000,12.951159,14.0
2,1102,396,1109,0.357078,70.972222,59.827489,11.144733,11.0
3,1102,396,1109,0.357078,70.972222,59.827489,11.144733,13.0
4,1103,692,1177,0.587935,76.354046,64.546392,11.807654,15.0


### 🔗 Merging Team Stats into Tournament Data
We now merge **team performance metrics** into **tournament match results**,  
so we can compare stats of both competing teams.


In [9]:
# Merge team stats for the winning team (WTeamID)
tourney_results = tourney_results.merge(team_stats, left_on="WTeamID", right_on="WTeamID", how="left")
tourney_results.rename(columns={
    "WinPercentage": "W_WinPercentage",
    "AvgPointsScored": "W_AvgPointsScored",
    "AvgPointsAllowed": "W_AvgPointsAllowed",
    "PointDifferential": "W_PointDifferential",
    "SeedValue": "W_SeedValue"
}, inplace=True)

# Merge team stats for the losing team (LTeamID)
tourney_results = tourney_results.merge(team_stats, left_on="LTeamID", right_on="WTeamID", how="left")
tourney_results.rename(columns={
    "WinPercentage": "L_WinPercentage",
    "AvgPointsScored": "L_AvgPointsScored",
    "AvgPointsAllowed": "L_AvgPointsAllowed",
    "PointDifferential": "L_PointDifferential",
    "SeedValue": "L_SeedValue"
}, inplace=True)

# Drop redundant WTeamID_y column
tourney_results.drop(columns=["WTeamID_y"], inplace=True, errors="ignore")

# Display updated tournament dataset
display(tourney_results.head())


,Season,DayNum,WTeamID_x,WScore,LTeamID,LScore,WLoc,NumOT,Wins_x,TotalGames_x,...,W_AvgPointsAllowed,W_PointDifferential,W_SeedValue,Wins_y,TotalGames_y,L_WinPercentage,L_AvgPointsScored,L_AvgPointsAllowed,L_PointDifferential,L_SeedValue
0,1985,136,1116,63,1234,54,N,0,811,1269,...,67.777293,16.204212,9.0,753,1254,0.600478,81.065073,67.538922,13.526151,8.0
1,1985,136,1116,63,1234,54,N,0,811,1269,...,67.777293,16.204212,9.0,753,1254,0.600478,81.065073,67.538922,13.526151,11.0
2,1985,136,1116,63,1234,54,N,0,811,1269,...,67.777293,16.204212,9.0,753,1254,0.600478,81.065073,67.538922,13.526151,2.0
3,1985,136,1116,63,1234,54,N,0,811,1269,...,67.777293,16.204212,9.0,753,1254,0.600478,81.065073,67.538922,13.526151,5.0
4,1985,136,1116,63,1234,54,N,0,811,1269,...,67.777293,16.204212,9.0,753,1254,0.600478,81.065073,67.538922,13.526151,4.0


### 🔀 Creating Feature Differences
Instead of using raw team statistics, we calculate **relative differences**:
- **WinPercentage_Diff** = (Winning Team % - Losing Team %)
- **AvgPointsScored_Diff** = (Winning Team Avg Points - Losing Team Avg Points)
- **SeedValue_Diff** = (Winning Team Seed - Losing Team Seed)


In [11]:
# Compute feature differences
tourney_results["WinPercentage_Diff"] = tourney_results["W_WinPercentage"] - tourney_results["L_WinPercentage"]
tourney_results["AvgPointsScored_Diff"] = tourney_results["W_AvgPointsScored"] - tourney_results["L_AvgPointsScored"]
tourney_results["AvgPointsAllowed_Diff"] = tourney_results["W_AvgPointsAllowed"] - tourney_results["L_AvgPointsAllowed"]
tourney_results["PointDifferential_Diff"] = tourney_results["W_PointDifferential"] - tourney_results["L_PointDifferential"]
tourney_results["SeedValue_Diff"] = tourney_results["W_SeedValue"] - tourney_results["L_SeedValue"]

# Select relevant features
model_data = tourney_results[[
    "WinPercentage_Diff",
    "AvgPointsScored_Diff",
    "AvgPointsAllowed_Diff",
    "PointDifferential_Diff",
    "SeedValue_Diff"
]].copy()

# Add target variable (1 = Winning team won)
model_data["Result"] = 1

# Display final dataset
display(model_data.head())


,WinPercentage_Diff,AvgPointsScored_Diff,AvgPointsAllowed_Diff,PointDifferential_Diff,SeedValue_Diff,Result
0,0.038607,2.916431,0.23837,2.678061,1.0,1
1,0.038607,2.916431,0.23837,2.678061,-2.0,1
2,0.038607,2.916431,0.23837,2.678061,7.0,1
3,0.038607,2.916431,0.23837,2.678061,4.0,1
4,0.038607,2.916431,0.23837,2.678061,5.0,1
